In [1]:
import os

os.environ["SEED"] = "0"
import numpy as np
import pandas as pd
from ConfigSpace import Configuration

from src.aac.AAC import AAC
from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, SEED
from src.database import DB
from src.database.queries import *
from src.instance.Instance import Instance
from src.instance.InstanceList import InstanceList
from src.instance.TSP_Instance import TSP_Instance, TSP_train_test_from_index_file
from src.solver.Portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.aac.SurrogateEstimator import Estimator1, SurrogateEstimator

In [2]:
train_instances, test_instances = TSP_train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=3,
)
train_instances

InstanceList(size=3)[TSP_Instance(filepath=TSP/CEPS_benchmark/explosion/21.tsp), TSP_Instance(filepath=TSP/CEPS_benchmark/rotation/34.tsp), TSP_Instance(filepath=TSP/CEPS_benchmark/compression/09.tsp)]

In [3]:
portfolio = Portfolio.from_solver_class(TSP_LKH_Solver, size=1)
portfolio

Portfolio(size=1)[Solver(id=52980316141715150)]

In [4]:
portfolio.evaluate(
    train_instances,
    prefix="test1",
    calculate_features=False,
    cache=False,
)

[2024-12-28 21:51:44] DEBUG     Portfolio.evaluate(test1)
[2024-12-28 21:51:44] DEBUG     Portfolio(size=1)[Solver(id=52980316141715150)]
[2024-12-28 21:51:44] DEBUG     solve(prefix=test1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/explosion/21.tsp))
[2024-12-28 21:51:44] DEBUG     solve(prefix=test1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/rotation/34.tsp))
[2024-12-28 21:51:44] DEBUG     solve(prefix=test1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/compression/09.tsp))
[2024-12-28 21:51:56] DEBUG     Solver.Result(prefix=test1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/rotation/34.tsp), cost=100.00, time=10.00, cached=False, surrogate=False, error=False)
[2024-12-28 21:52:01] DEBUG     Solver.Result(prefix=test1, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/explos

Portfolio.Result(prefix=test1, cost=100.00, time=[30.00])

In [5]:
portfolio.evaluate(
    train_instances,
    prefix="test2",
    calculate_features=False,
    cache=False,
)

[2024-12-28 21:52:26] DEBUG     Portfolio.evaluate(test2)
[2024-12-28 21:52:26] DEBUG     Portfolio(size=1)[Solver(id=52980316141715150)]
[2024-12-28 21:52:26] DEBUG     solve(prefix=test2, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/explosion/21.tsp))
[2024-12-28 21:52:26] DEBUG     solve(prefix=test2, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/rotation/34.tsp))
[2024-12-28 21:52:26] DEBUG     solve(prefix=test2, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/compression/09.tsp))
[2024-12-28 21:52:38] DEBUG     Solver.Result(prefix=test2, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/rotation/34.tsp), cost=100.00, time=10.00, cached=False, surrogate=False, error=False)
[2024-12-28 21:52:39] DEBUG     Solver.Result(prefix=test2, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/explos

Portfolio.Result(prefix=test2, cost=100.00, time=[30.00])

[2024-12-28 21:53:03] DEBUG     Solver.Result(prefix=test2, solver=Solver(id=52980316141715150), instance=TSP_Instance(filepath=TSP/CEPS_benchmark/compression/09.tsp), cost=100.00, time=10.00, cached=False, surrogate=False, error=False)


In [5]:
aac = AAC(
    portfolio=portfolio,
    instance_list=train_instances,
    prefix="config",
    t_c=100,
    calculate_features=False,
    estimator=None,
)

InstanceList(size=3)[TSP_Instance(filepath=TSP/CEPS_benchmark/explosion/21.tsp), TSP_Instance(filepath=TSP/CEPS_benchmark/rotation/34.tsp), TSP_Instance(filepath=TSP/CEPS_benchmark/compression/09.tsp)]

In [6]:
db = DB()
# db = DB("database/run-test_924023.db")

In [7]:
pd.read_sql_query("SELECT * FROM instances", db._conn)

,id,filepath,optimum
0,855157651668393360,TSP/CEPS_benchmark/explosion/21.tsp,14928292.0
1,1353678304570487518,TSP/CEPS_benchmark/rotation/34.tsp,15486460.0
2,810870798045289410,TSP/CEPS_benchmark/compression/09.tsp,16003060.0


In [8]:
pd.read_sql_query("SELECT * FROM solvers", db._conn)

,id,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,GAIN23,GAIN_CRITERION,INITIAL_STEP_SIZE,...,MAX_BREADTH,MAX_CANDIDATES,MOVE_TYPE,PATCHING_A,PATCHING_C,POPULATION_SIZE,RESTRICTED_SEARCH,SUBGRADIENT,SUBSEQUENT_MOVE_TYPE,SUBSEQUENT_PATCHING
0,52980316141715150,0.6,1.0,0.0,0.0,0.7,0.0,1.0,0.0,0.0,...,0.444643,1.0,0.25,0.6,0.2,0.061224,0.0,0.0,5.0,1.0


In [9]:
results = pd.read_sql_query("SELECT * FROM results", db._conn)
results

,id,prefix,solver_id,instance_id,cost,time,cached,surrogate,error
0,test1_52980316141715150_1353678304570487518,test1,52980316141715150,1353678304570487518,100.0,10.0,0,0,0
1,test1_52980316141715150_855157651668393360,test1,52980316141715150,855157651668393360,100.0,10.0,0,0,0
2,test1_52980316141715150_810870798045289410,test1,52980316141715150,810870798045289410,100.0,10.0,0,0,0
3,test2_52980316141715150_1353678304570487518,test2,52980316141715150,1353678304570487518,100.0,10.0,0,0,0
4,test2_52980316141715150_855157651668393360,test2,52980316141715150,855157651668393360,100.0,10.0,0,0,0
5,test2_52980316141715150_810870798045289410,test2,52980316141715150,810870798045289410,100.0,10.0,0,0,1


In [ ]:
pd.read_sql_query("SELECT * FROM evaluations", db._conn)